# Neural Radiance Fields

### What Problem are we trying to solve?
#### View Synthesis

### what are Neural Randiance Fields?

- 3d neural network encoding for continous viewpoints
we want to over fit a neural network to a single point- 
- The weights of the Neural Networks represent the Scene


#### Positional Encoding


#### Represent a scene as a continous 5D function 

***Input:***
- $$(x,y,z,\theta, \phi)$$
    - $$(x,y,z) - Coordinates$$
    - $$(\theta, \phi) - viewing\;direction$$

***Output:***
- $$(r,g,b,\sigma)$$
    - $$(r,g,b) - color\;channels$$
    - $$\sigma - density$$


## Import Statements

In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import matplotlibas plt
